# Usare Zenodo come Research Data Manager (RDM)

In questo Notebook vedremo come poter utilizzare un RDM con alcuni esempi
Il nostro repository ufficiale si trova su 

https://www.openaccessrepository.it

ma per questo notebook useremo un ambiente di test, raggiungibile all'indirizzo

https://sandbox.zenodo.org

Impostiamo dunque una variabile che punta al repository che intendiamo utilizzare:

In [1]:

oar_host = "www.openaccessrepository.it"

cern_host = "sandbox.zenodo.org"


HOST = oar_host

Per ottenere i permessi di scrittura sul repository, è necessario creare un token dal portale web del repository.
Nel caso di Zenodo, lo possiamo ottenere su:

https://sandbox.zenodo.org/account/settings/applications/

Se si utilizza invece il repository di produzione:

https://www.openaccessrepository.it/account/settings/applications/

Creare un nuovo *Personal Access Tokens*
e configurare la variabile TOKEN



In [77]:
import requests

### OAR.it token
oar_token = "WctVavtQvHXupfUTYSkPaieQgATwbRyvVFO6KnsjsC8JWr9N40IZLIdgbbGQ"

### Zenodo sandbox token
cern_token = "XveBvDXazblMrQeoiExquJUKUjP60S5UyQyK4Kj04CnseRwqwvSTYxt90Nw9"

TOKEN = oar_token



200

In [ ]:
# Elenco dei nostri attuali depositi
api = "/api/deposit/depositions"
deposit_url = f'https://{HOST}{api}'
req = requests.get(deposit_url, params={'access_token': TOKEN})
req.status_code


In [ ]:
req.text


## Carichiamo i record dal nostro file YAML di input

In [95]:
import yaml

with open(r'adone-prova1.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    records = yaml.load(file, Loader=yaml.FullLoader)


Controlliamo quanti record sono stati caricati

In [81]:
len(records)

#records[3]

2

Creiamo un nuovo deposito, in modo da ottenere un deposit identifier

In [83]:
headers = {"Content-Type": "application/json"}
params = {'access_token': TOKEN}
api = "/api/deposit/depositions"
deposit_url = f'https://{HOST}{api}'

req = requests.post(deposit_url, params=params, json={})
                   # Headers are not necessary here since "requests" automatically
                   # adds "Content-Type: application/json", because we're using
                   # the "json=" keyword argument
                   # headers=headers,
                   # headers=headers)


print(req.status_code)

req.json()


201


{'conceptrecid': '75348',
 'created': '2022-03-25T11:53:42.999700+00:00',
 'files': [],
 'id': 75349,
 'links': {'bucket': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97',
  'discard': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/discard',
  'edit': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/edit',
  'files': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/files',
  'html': 'https://www.openaccessrepository.it/deposit/75349',
  'latest_draft': 'https://www.openaccessrepository.it/api/deposit/depositions/75349',
  'latest_draft_html': 'https://www.openaccessrepository.it/deposit/depositions/75349',
  'publish': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/publish',
  'self': 'https://www.openaccessrepository.it/api/deposit/depositions/75349'},
 'metadata': {'prereserve_doi': {'doi': '10.15161/oar.it/75349',
   'recid': 75349}},
 'modified': 

Il nuovo deposito ha assegnato un `id` e riserva anche un DOI (generato localmente, non registrato su datacite)

In [84]:
deposition_id = req.json()['id']
reserved_doi = req.json()['metadata']['prereserve_doi']['doi']

deposition_id, reserved_doi

(75349, '10.15161/oar.it/75349')

Il nuovo deposito ci restituisce anche un bucket su cui caricare i nostri files

In [86]:
bucket_url = req.json()["links"]["bucket"]
bucket_url



'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97'

Selezioniamo un record dal nostro file `.yml`

In [107]:
r = records[0]

r

{'title': 'Phylosophy and cost of the injection system for CARST',
 'description': '<p>The general specifications for any synchroton light source injection system are: full Main ring (MR) energy, MR filling time to maximum foreseen current, much shorter than the beam lifetime. (...) Full energy injection guarantees that the problems to arise, in connection with the need for keeping a large number of beam lines independently aligned to very high precision, will be reduced to a minimum.</p>',
 'access_right': 'open',
 'communities': [{'identifier': 'infn'}],
 'creators': [{'name': 'Tazzari, S.'}],
 'keywords': ['ADONE', 'CARST', 'Injector systems'],
 'language': 'eng',
 'license': {'id': 'CC-BY-4.0'},
 'publication_date': '1986-07-27',
 'resource_type': {'type': 'publication', 'subtype': 'proposal'},
 'files': [{'key': 'AF-09.pdf', 'path': 'AF_1/AF-09.pdf'}]}

il nome del file su OAR sarà il titolo in cui al posto degli spazi sostituiamo il simbolo '-' (dash) seguito dal percorso `path` del file, seguito dall'estensione

In [109]:

filename = r['title'].replace(' ', '-').lower() 
filename += '_' + r['files'][0]['path'].replace('/', '_')
path = r['files'][0]['path'] 

import urllib.parse


path, filename, urllib.parse.quote(filename)

('AF_1/AF-09.pdf',
 'phylosophy-and-cost-of-the-injection-system-for-carst_AF_1_AF-09.pdf',
 'phylosophy-and-cost-of-the-injection-system-for-carst_AF_1_AF-09.pdf')

Facciamo l'upload del file sulla nostra deposition, caricandolo da una cartella locale, usando il path indicato nel file `.yml` di input

In [92]:
#filename = "phylosophy-and-cost-of-the-injection-system-for-carst.pdf"  # name of the file on oar

params = {'access_token': TOKEN}
upload_url = f'{bucket_url}/{filename}'
print(upload_url)

# The target URL is a combination of the bucket link with the desired filename
# seperated by a slash.

with open(path, "rb") as fp:
    req = requests.put(
        upload_url,
        data=fp,
        params=params,
    )
req.json()

https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf


{'delete_marker': False,
 'mimetype': 'application/pdf',
 'updated': '2022-03-25T12:04:00.360206+00:00',
 'links': {'self': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf',
  'version': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf?versionId=38957e8e-2536-4e6e-afac-c5b7959e6bb4',
  'uploads': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf?uploads'},
 'checksum': 'md5:a7d9fa61c43baac4fdb51ac1a7b4e55e',
 'is_head': True,
 'created': '2022-03-25T12:03:59.740215+00:00',
 'size': 4098936,
 'version_id': '38957e8e-2536-4e6e-afac-c5b7959e6bb4',
 'key': 'the-afrodite-booster_AF_1_AF-02.pdf'}

Selezioniamo i metadati da caricare. Ad esempio il primo record:

In [93]:


#ec['communities'][0]['identifier'] = rec['communities'][0].pop('id')
r


{'title': 'The AFRODITE booster',
 'description': '<p>The booster ring for the AFRODITE injector chain has been studied to achieve a low emittance (...) accepting an injected beam from a positron linac without an intermediate positron accumulator. The lattice has been designated using the same magnetic elements of the main ring, bearing in mind that substancial cost reduction can be obtained with this approach.</p>',
 'access_right': 'open',
 'communities': [{'identifier': 'infn'}],
 'creators': [{'name': 'Patteri, P.'}, {'name': 'Tazzioli, F.'}],
 'keywords': ['ADONE', 'AFRODITE', 'Booster ring'],
 'language': 'eng',
 'license': 'CC-BY-4.0',
 'publication_date': datetime.date(1986, 6, 26),
 'upload_type': 'publication',
 'publication_type': 'technicalnote',
 'files': [{'key': 'AF-02.pdf', 'path': 'AF_1/AF-02.pdf'}]}

Prepariamo i metadati facendo un po' di "massaging" sui nomi degli attributi, rimuovendo gli attributi non più in uso. **da rivedere i nomi degli attributi** del file `.yml` e carichiamo i metadati

In [97]:


# metadata = {
#     'upload_type': rec['resource_type'].pop('type'),
#     'publication_type': rec['resource_type'].pop('subtype')
# }
# del rec['resource_type']
del r['files']

#deposition_id = 75345

data = { 'metadata': r }
print(data)
import json

url = f'{deposit_url}/{deposition_id}'
req = requests.put(url,                  
    params=params, data=json.dumps(data),
    headers=headers)

req.json()


{'metadata': {'title': 'The AFRODITE booster', 'description': '<p>The booster ring for the AFRODITE injector chain has been studied to achieve a low emittance (...) accepting an injected beam from a positron linac without an intermediate positron accumulator. The lattice has been designated using the same magnetic elements of the main ring, bearing in mind that substancial cost reduction can be obtained with this approach.</p>', 'access_right': 'open', 'communities': [{'identifier': 'infn'}], 'creators': [{'name': 'Patteri, P.'}, {'name': 'Tazzioli, F.'}], 'keywords': ['ADONE', 'AFRODITE', 'Booster ring'], 'language': 'eng', 'license': 'CC-BY-4.0', 'publication_date': '1986-06-26', 'upload_type': 'publication', 'publication_type': 'technicalnote'}}


{'conceptrecid': '75348',
 'created': '2022-03-25T11:53:42.999700+00:00',
 'files': [{'checksum': 'a7d9fa61c43baac4fdb51ac1a7b4e55e',
   'filename': 'the-afrodite-booster_AF_1_AF-02.pdf',
   'filesize': 4098936,
   'id': '06b3de7c-9214-4f16-896f-32e4306a4a58',
   'links': {'download': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf',
    'self': 'https://www.openaccessrepository.it/api/deposit/depositions/75343/files/06b3de7c-9214-4f16-896f-32e4306a4a58'}}],
 'id': 75349,
 'links': {'bucket': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97',
  'discard': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/discard',
  'edit': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/edit',
  'files': 'https://www.openaccessrepository.it/api/deposit/depositions/75349/files',
  'html': 'https://www.openaccessrepository.it/deposit/75349',
  'la

E se tutto è andato bene possiamo pubblicare il nostro record:

In [103]:
publish_url = f'{deposit_url}/{deposition_id}/actions/publish'
req = requests.post(publish_url, params=params )

publish_url, req.status_code, req.json()


('https://www.openaccessrepository.it/api/deposit/depositions/75349/actions/publish',
 202,
 {'conceptdoi': '10.15161/oar.it/75348',
  'conceptrecid': '75348',
  'created': '2022-03-25T11:53:42.999700+00:00',
  'doi': '10.15161/oar.it/75349',
  'doi_url': 'https://doi.org/10.15161/oar.it/75349',
  'files': [{'checksum': 'a7d9fa61c43baac4fdb51ac1a7b4e55e',
    'filename': 'the-afrodite-booster_AF_1_AF-02.pdf',
    'filesize': 4098936,
    'id': '06b3de7c-9214-4f16-896f-32e4306a4a58',
    'links': {'download': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97/the-afrodite-booster_AF_1_AF-02.pdf',
     'self': 'https://www.openaccessrepository.it/api/deposit/depositions/75310/files/06b3de7c-9214-4f16-896f-32e4306a4a58'}}],
  'id': 75349,
  'links': {'badge': 'https://www.openaccessrepository.it/badge/doi/10.15161/oar.it/75349.svg',
   'bucket': 'https://www.openaccessrepository.it/api/files/50118fab-1336-48c9-ae18-fc9c4fe47e97',
   'conceptbadge': 'https: